In [64]:
import pandas as pd
import difflib
import numpy as np
import re

In [65]:
data = pd.read_csv('/Users/christophermccarthy/Documents/datathon_2021/Purchase_Card_Transactions.csv')
ranges = pd.read_csv('/Users/christophermccarthy/Documents/datathon_2021/iso18245_ranges.csv')
mcc_codes = pd.read_csv('/Users/christophermccarthy/Documents/datathon_2021/visa_list.csv')

In [66]:
# GET GROUPS

ranges.index = pd.IntervalIndex.from_arrays(ranges['MCC START'],ranges['MCC END'],closed='both')
mcc_codes['GROUP'] = mcc_codes['MCC'].apply(lambda x : ranges.iloc[ranges.index.get_loc(x)]['DESCRIPTION']
                                           )

In [67]:
#Lower Case
def strip_punc(x):
    x = str(x)
    x = x.replace("&", 'and')
    x = re.sub(r'[^\w\s]','',x)
    x = x.replace(" ", '')
    x = x.replace("shop", 'store')
    return x

mcc_codes['DESCRIPTION'] = mcc_codes.DESCRIPTION.str.lower()
data['MCC_DESCRIPTION'] = data.MCC_DESCRIPTION.str.lower()

mcc_codes['DESCRIPTION'] = mcc_codes.DESCRIPTION.apply(lambda x : strip_punc(x))
data['MCC_DESCRIPTION'] = data.MCC_DESCRIPTION.apply(lambda x : strip_punc(x))

mcc_codes = mcc_codes.set_index('DESCRIPTION')


In [68]:
# Get close matches
def fix_shitty_format(s):
    ret_dict = {}
    for code in s:
        try: 
            close_matches = difflib.get_close_matches(code, mcc_codes.index)
            try:
                ret_dict[code] = close_matches[0]
            except:
                ret_dict[code] = ''
        except:
            ret_dict[code] = ''
    return ret_dict

fix_dict = fix_shitty_format(data.MCC_DESCRIPTION.unique())
data['join_key'] = data.MCC_DESCRIPTION.apply(lambda x : fix_dict[x])

In [69]:
data_fixed = data.join(mcc_codes, on = 'join_key')

In [70]:
display(data_fixed)

,OBJECTID,AGENCY,TRANSACTION_DATE,TRANSACTION_AMOUNT,VENDOR_NAME,VENDOR_STATE_PROVINCE,MCC_DESCRIPTION,join_key,MCC,REQUIRED NAME IN AUTHORIZATION REQUEST / CLEARING RECORD,GROUP
0,2,Department of Mental Health,2009/01/05 05:00:00+00,229.50,WW GRAINGER 912,DC,industrialsuppliesnotelsewhereclassified,industrialsuppliesnotelsewhereclassified,5085.0,NaN,Retail outlets
1,3,District Department of Transportation,2009/01/05 05:00:00+00,3147.33,BRANCH SUPPLY,DC,stationeryofficeandschoolsupplystores,stationerystoresofficeandschoolsupplystores,5943.0,NaN,Miscellaneous outlets
2,4,Department of Youth Rehabilitation Services,2009/01/05 05:00:00+00,54.00,VIDEO MONITORING SRVCS,NY,motionpicturesvideotapeproductionanddistribution,motionpictureandvideotapeproductionanddistribu...,7829.0,NaN,Amusement and entertainment
3,5,Office of Employee Appeals,2009/01/05 05:00:00+00,137.35,STAPLES 00102186,DC,stationeryofficesuppliesprintingwritingpaper,stationeryofficesuppliesprintingandwritingpaper,5111.0,NaN,Retail outlets
4,6,Office of Police Complaints,2009/01/06 05:00:00+00,1340.80,FAITH MGMT CONSULTING,DC,stationeryofficeandschoolsupplystores,stationerystoresofficeandschoolsupplystores,5943.0,NaN,Miscellaneous outlets
...,...,...,...,...,...,...,...,...,...,...,...
433921,433922,Unknown New Agency,2020/10/26 04:00:00+00,480.00,IN *BROADCAST ELECTRON,IL,electricalpartsandequipment,electricalpartsandequipment,5065.0,NaN,Retail outlets
433922,433923,Unknown New Agency,2020/10/26 04:00:00+00,2112.00,IN *BROADCAST ELECTRON,IL,electricalpartsandequipment,electricalpartsandequipment,5065.0,NaN,Retail outlets
433923,433924,Metropolitan Police Department,2020/10/26 04:00:00+00,3689.01,GLORY GLOBAL SOLUTIONS,IL,commercialequipmentnotelsewhereclassified,commercialequipmentnotelsewhereclassified,5046.0,NaN,Retail outlets
433924,433925,Unknown New Agency,2020/10/26 04:00:00+00,65.70,DEDOOSE,CA,professionalservicesnotelsewhereclassified,professionalservicesnotelsewhereclassified,8999.0,NaN,Professional services and membership organizat...


In [42]:
data_fixed.GROUP.value_counts()

Miscellaneous outlets                                 114481
Retail outlets                                        104437
Professional services and membership organizations     47959
Business services                                      36352
Transportation                                         29908
Reserved for private use                               24548
Government services                                    11366
Contracted services                                    11249
Amusement and entertainment                             9490
Utilities                                               8802
Service providers                                       5837
Repair services                                         3032
Clothing outlets                                        2842
Agricultural services                                    446
Name: GROUP, dtype: int64

In [71]:
display(data_fixed[data_fixed.GROUP.isnull()].MCC_DESCRIPTION.value_counts())

recordstoresandmusicdeliveredviatheinternet     353
schoolsvocational                               351
unknown                                         323
limousinesandtaxicabs                           286
equipmentrentalleasingsvsfurnturetoolrental     190
schoolstradeandvocational                       136
telecominclprepaidrecurringphonesvcs            105
publicwarehousingfarmrefriggoodshhgstorage       89
sportspromotersandprofessional                   30
organizationspolitical                           27
debtmarriagepersonalcounselingservice            26
theinngroup                                      26
sanitationpolishingspecialitycleaningprep        24
clubscntrymbrshipathletrecsprtsprivategolf       21
alterationsmendingseamstressestailors            18
fueldispenserautomated                           17
unknownvendor                                    16
aubergewandylinn                                 11
games                                            10
athlticfield